### INST447 Project Part 2
#### Members: Alexander Chui and Matthew Chan

### What factors affect ratings for an anime?

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importing the required libraries
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen 
import numpy as np
from requests import get

In [3]:
#libraries imported for handling rate limits and console formatting
from time import time, sleep
from random import randint
from warnings import warn
from IPython.core.display import clear_output

In [4]:
import regex as re

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis

import nltk; 

C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar'

In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
#Connecting to the URL on the web to access its content (Crunchyroll)
# html=urlopen('https://www.crunchyroll.com/fullmetal-alchemist-brotherhood/reviews/helpful/page1')
# html.status

In [7]:
#Connecting to the URL on the web to access its content (MAL)
html=urlopen('https://myanimelist.net/anime/6205/K%C3%A4mpfer/reviews?p=1')
html.status

200

In [8]:
#Creating an instance of beautifulsoup object to begin web scraping
bs=BeautifulSoup(html.read(),'html.parser')

In [9]:
#Acessing the container objects with the anime info 
anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})
print(len(anime_review_containers))

20


In [10]:
#Creating empty lists to store all the items 
review = []

In [11]:
# Preparing the monitoring of the loop
start_time = time()
request = 0

In [12]:
#Extracting each job name in loop
for page in range(1,5):
    # Make a get request
        #https://myanimelist.net/anime/genre/1/Action?page=1
        #https://myanimelist.net/anime/genre/1/Action?page=2
        response = get('https://myanimelist.net/anime/6205/K%C3%A4mpfer/reviews?p='+ str(page))
        
#         # Pause the loop
#         sleep(randint(8,15))

#         # Monitor the requests
#         request += 1
#         elapsed_time = time() - start_time
#         print('Request:{}; Frequency: {} requests/s'.format(request, 
#                                                             request/elapsed_time))
#         clear_output(wait = True)

#         # Throw a warning for non-200 status codes
#         if response.status_code != 200:
#             warn('Request: {}; Status code: {}'.format(request, 
#                                                        response.status_code))

#         # Break the loop if the number of requests is greater than expected
#         if request > 100:
#             warn('Number of requests was greater than expected.')
#             break
        
        # Parse the content of the request with BeautifulSoup
        bs = BeautifulSoup(response.text, 'html.parser')
        
        #Acessing the container objects with the anime info 
        anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})

        #Extracting all the elements 
        for container in anime_review_containers:
        # If the movie has Metascore, then extract:
        #  if container.find('div', {'class':{'scoremem'}})  is not None:
        
        # The name of anime
            try: 
                reviews = container.find('div', {'class':'spaceit textReadability word-break pt8 mt8'})
                review.append(reviews.text)
            except:
                review.append('Missing')

In [13]:
#Creating a pandas dataframe to store the objects extracted
anime_review = pd.DataFrame({'Review': review
})
print(anime_review.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  80 non-null     object
dtypes: object(1)
memory usage: 768.0+ bytes
None


In [14]:
# Display the top 5 rows of dataset
anime_review.head()

,Review
0,\n\n\n\nOverall\n5\n\n\nStory\n5\n\n\nAnimatio...
1,\n\n\n\nOverall\n7\n\n\nStory\n7\n\n\nAnimatio...
2,\n\n\n\nOverall\n6\n\n\nStory\n6\n\n\nAnimatio...
3,\n\n\n\nOverall\n7\n\n\nStory\n5\n\n\nAnimatio...
4,\n\n\n\nOverall\n8\n\n\nStory\n8\n\n\nAnimatio...


In [15]:
# Convert to list
data = anime_review['Review'].tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

print(data[0])

 Overall 5 Story 5 Animation 6 Sound 9 Character 4 Enjoyment 6 I really love all of the crazy implausible story concepts anime is capable of delivering us. Kämpfer is no exception, as it combines many time honored and tested themes of mahou shoujo-esque transformations, girls fighting each other, gender confusion, and lesbianism. Mix in plenty of service and a clichéd harem of the usual personality types and you have an anime that basically been made a hundred times before. And a lot better too. Even though, I still found myself inexorability drawn to this title because of its fabulous seiyuu cast. The premise for Kämpfer is a bit unique. While it shares many of the same themes as shows like Ikkitousen and Sekirei, Kämpfer deviates by essentially making the male lead one of his own harem by forcing him to transform into a girl in order to participate in the battles. The girls are divided into factions, red and blue, and must battle each other to the death (apparently) at the behest of 

<>:5: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \s
<ipython-input-15-b412c0db5fd7>:5: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


In [16]:
# Clean and tokenize the reviews

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data_words = list(sent_to_words(data))

print(data_words[0])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'really', 'love', 'all', 'of', 'the', 'crazy', 'implausible', 'story', 'concepts', 'anime', 'is', 'capable', 'of', 'delivering', 'us', 'kampfer', 'is', 'no', 'exception', 'as', 'it', 'combines', 'many', 'time', 'honored', 'and', 'tested', 'themes', 'of', 'mahou', 'shoujo', 'esque', 'transformations', 'girls', 'fighting', 'each', 'other', 'gender', 'confusion', 'and', 'lesbianism', 'mix', 'in', 'plenty', 'of', 'service', 'and', 'cliched', 'harem', 'of', 'the', 'usual', 'personality', 'types', 'and', 'you', 'have', 'an', 'anime', 'that', 'basically', 'been', 'made', 'hundred', 'times', 'before', 'and', 'lot', 'better', 'too', 'even', 'though', 'still', 'found', 'myself', 'inexorability', 'drawn', 'to', 'this', 'title', 'because', 'of', 'its', 'fabulous', 'seiyuu', 'cast', 'the', 'premise', 'for', 'kampfer', 'is', 'bit', 'unique', 'while', 'it', 'shares', 'many', 'of', 'the', 'same', 'themes', 'as', 'shows', 'like', 'ikk

In [17]:
# Create bigrams, remove stop words and lemmatize the reviews

# Build the bigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

print(bigram_mod[data_words[4]])

['overall', 'story', 'animation_sound', 'character', 'enjoyment', 'review', 'after', 'reading', 'manga', 'young', 'male', 'love', 'girl', 'at', 'his', 'school', 'but', 'doesnt', 'have', 'the', 'guts', 'to', 'ask', 'her', 'out', 'yet', 'so', 'has', 'been', 'keeping', 'it', 'at', 'friend', 'level', 'but', 'suddenly', 'hes', 'thrown', 'into', 'game', 'that', 'can', 'cost', 'him', 'his', 'life', 'but', 'comes', 'with', 'bonus', 'that', 'can', 'get', 'him', 'closer', 'to', 'the', 'girls', 'of', 'his', 'dreams', 'sure', 'this', 'sounds', 'like', 'million', 'other', 'anime', 'about', 'guys', 'who', 'cant', 'get', 'their', 'act', 'to', 'gether', 'but', 'whats', 'so', 'wrong', 'with', 'that', 'love', 'this', 'kinda', 'storys', 'and', 'as', 'long', 'as', 'theres', 'twist', 'why', 'would', 'complain', 'about', 'the', 'same', 'old', 'same', 'old', 'build', 'if', 'you', 'dont', 'enjoy', 'the', 'anime', 'where', 'guys', 'become', 'girls', 'and', 'try', 'to', 'hook', 'up', 'with', 'their', 'dream', '

In [18]:
# Defining removing stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Defining making bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Defining lemmatization
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

['overall', 'story', 'character', 'enjoyment', 'really', 'love', 'crazy', 'implausible', 'story', 'concept', 'anime', 'capable', 'deliver', 'kampfer', 'exception', 'combine', 'many', 'time', 'honor', 'test', 'theme', 'shoujo', 'esque', 'transformation', 'girl', 'fight', 'gender', 'confusion', 'lesbianism', 'mix', 'plenty', 'service', 'cliche', 'harem', 'usual', 'personality', 'type', 'anime', 'basically', 'make', 'time', 'lot', 'well', 'even', 'still', 'find', 'inexorability', 'draw', 'title', 'fabulous', 'seiyuu', 'cast', 'premise', 'kampfer', 'bit', 'unique', 'share', 'many', 'theme', 'show', 'ikkitousen', 'sekirei', 'kampfer', 'deviate', 'essentially', 'make', 'male', 'lead', 'harem', 'force', 'transform', 'girl', 'order', 'participate', 'battle', 'girl', 'divide', 'faction', 'red', 'blue', 'battle', 'death', 'apparently', 'behest', 'mysterious', 'sinister', 'sound', 'faction', 'know', 'moderator', 'start', 'seem', 'set', 'series', 'chalk', 'full', 'exciting', 'action', 'ecchi', 'ge

In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 6), (4, 5), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 2), (13, 2), (14, 5), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 5), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 1), (45, 4), (46, 1), (47, 7), (48, 1), (49, 1), (50, 2), (51, 1), (52, 2), (53, 1), (54, 1), (55, 3), (56, 1), (57, 2), (58, 5), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 3), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 2), (82, 1), (83, 1), (84, 1), (85, 4), (86, 1), (87, 2), (88, 1), (89, 1), (90, 1), (91, 3), (92, 1), (93, 1), (94, 3), (95, 1), (96, 1), (97, 1), (98, 9), (99, 1), (100, 2), (101, 2), (102, 1), (103, 1), (104, 3), (105, 2), (106, 3), (107, 2), (108, 3), (109, 1), (110, 1),

In [21]:
# Using LDA to develop a topic model with 8 models

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
# Print the KeywordS in the 8 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.023*"character" + 0.023*"kampfer" + 0.019*"anime" + 0.017*"story" + 0.015*"girl" + 0.010*"natsuru" + 0.009*"really" + 0.009*"watch" + 0.009*"get" + 0.009*"overall"'), (1, '0.026*"character" + 0.022*"anime" + 0.017*"girl" + 0.013*"story" + 0.010*"kampfer" + 0.009*"really" + 0.009*"show" + 0.009*"plot" + 0.009*"get" + 0.008*"make"'), (2, '0.020*"kampfer" + 0.016*"character" + 0.016*"girl" + 0.015*"anime" + 0.013*"story" + 0.013*"natsuru" + 0.013*"show" + 0.012*"really" + 0.009*"get" + 0.008*"well"'), (3, '0.012*"service" + 0.009*"story" + 0.009*"anime" + 0.008*"well" + 0.008*"female" + 0.007*"know" + 0.007*"get" + 0.007*"kampfer" + 0.006*"episode" + 0.006*"overall"'), (4, '0.005*"switch" + 0.005*"bracelet" + 0.004*"unique" + 0.004*"normal" + 0.004*"sure" + 0.004*"point" + 0.004*"really" + 0.003*"want" + 0.003*"twelfth" + 0.003*"inclusion"'), (5, '0.037*"kampfer" + 0.013*"natsaru" + 0.009*"fight" + 0.009*"moderator" + 0.009*"force" + 0.008*"red" + 0.007*"blue" + 0.007*"struggle" +

In [22]:
# Generate a visualization of the topic models

import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.119599 -0.021229       1        1  50.215932
2     -0.112267 -0.015780       2        1  29.893487
1     -0.089222 -0.017653       3        1   8.843754
3      0.006300  0.080709       4        1   3.761493
7     -0.024333  0.033343       5        1   3.533064
5      0.105744 -0.125600       6        1   3.024562
4      0.113254  0.037041       7        1   0.472091
6      0.120122  0.029168       8        1   0.255618, topic_info=            Term        Freq       Total Category  logprob  loglift
152      kampfer  328.000000  328.000000  Default  30.0000  30.0000
47     character  321.000000  321.000000  Default  29.0000  29.0000
259         show  158.000000  158.000000  Default  28.0000  28.0000
14         anime  275.000000  275.000000  Default  27.0000  27.0000
238       really  153.000000  153.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
2324   recognize    0.111843    2.336001   Topic8  -5.9260   2.9301
1766        mood    0.092017    2.834192   Topic8  -6.1211   2.5417
2079         pay    0.122215    5.295155   Topic8  -5.8373   2.2005
824   interested    0.112553    9.389417   Topic8  -5.9197   1.5453
1408       break    0.103086    3.958360   Topic8  -6.0076   2.3212

[531 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1649      1  0.565577  achieve
2         1  0.591235      act
2         2  0.253387      act
2         6  0.168924      act
3         1  0.451870   action
...     ...       ...      ...
915       2  0.174752     wish
915       3  0.524256     wish
1860      1  0.567723    wrist
646       1  0.569496  writing
1161      1  0.893378      yet

[814 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 4, 8, 6, 5, 7])